# 과제 5: 배치 변환을 사용하여 대규모 데이터 집합에서 추론 결과 가져오기

## 과제 5.1: 환경 설정

패키지 및 종속성을 설치합니다.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import time
from sagemaker.session import Session
from sagemaker.transformer import Transformer

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
sagemaker_session = sagemaker.Session()
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

훈련 및 튜닝 실습에서 사용했던 모델을 기본 Amazon Simple Storage Service(Amazon S3) 버킷에 저장합니다. **create_model**을 사용하여 모델을 설정하고 훈련된 모델을 참조하도록 **ModelDataUrl**을 구성합니다.

In [ ]:
#set-up-model
# Upload the model and dataset to your Amazon S3 bucket
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

기본 Amazon S3 버킷에 배치 레코드를 업로드합니다.

In [ ]:
#upload-dataset
s3_client.upload_file(Filename="batch_data.csv", Bucket=bucket, Key=f"{prefix}/batch_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/batch_data.csv"

## 과제 5.2: 배치 변환 작업 생성

배치 변환에서는 지정한 파라미터의 한도 내에서 대규모 데이터 집합 처리를 자동으로 관리합니다. 배치 변환 작업이 시작되면 SageMaker는 컴퓨팅 인스턴스를 초기화하고 이러한 인스턴스 간에 추론 또는 전처리 워크로드를 분산합니다. 배치 변환에서는 키를 기준으로 입력의 Amazon S3 객체를 분할하고 인스턴스에 Amazon S3 객체를 매핑합니다.

대규모 데이터 집합에서 추론 결과를 가져와야 하거나 영구 엔드포인트가 필요하지 않을 때는 배치 변환을 사용하면 됩니다.

배치 변환 작업을 생성하려면 다음 옵션을 설정해야 합니다.
- **model_name**: 모델의 이름입니다.
- **instance_type**: 사용할 Amazon Elastic Compute Cloud(Amazon EC2) 인스턴스의 유형(예: ‘ml.c4.xlarge’)입니다.
- **instance_count**: 사용할 EC2 인스턴스의 수입니다.
- **assemble_with**: 출력이 조합되는 방식입니다. 유효한 값은 ‘Line’ 또는 ‘None’입니다.
- **strategy**: 단일 요청에서 레코드를 배치 처리하는 방법을 결정하는 데 사용되는 전략입니다. 유효한 값은 ‘MultiRecord’ 및 ‘SingleRecord’입니다.
- **accept**: 허용할 파일 형식입니다.
- **output_path**: 변환 결과를 저장할 Amazon S3 위치입니다. 값을 지정하지 않으면 결과는 기본 버킷에 저장됩니다.

In [ ]:
#create-batch-transformer
transformer = Transformer(
    model_name=model_name,
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

테스트 데이터 집합을 고객 레코드로 사용하여 배치 변환 작업을 실행합니다. 이 고객 레코드 집합을 사용하여 작업을 실행하려면 10분 정도 걸릴 수 있습니다.

배치 변환 작업에 관한 자세한 내용은 [배치 변환 사용](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)을 참조하세요.

In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

## 과제 5.3: Amazon S3에서 예측 데이터 확인

배치 변환 작업의 출력은 변환기 설정 시 지정한 버킷과 폴더에 저장됩니다. AWS Management Console이나 노트북에서 Amazon S3의 예측 결과를 확인할 수 있습니다.

콘솔에서 출력을 다운로드하여 확인하려는 경우 Amazon S3로 이동하여 이름이 **sagemaker-** 로 시작하는 버킷을 열고 **/sagemaker/mlasms/batch-transform/test**에 있는 객체로 이동합니다. **batch_data.csv.out** 객체를 다운로드하여 메모장 편집기에서 엽니다. 배치 변환 작업을 통해 실행한 고객 레코드에 대해 예측된 값 수백 개가 파일에 포함되어 있습니다.

노트북에서는 출력 샘플도 표시할 수 있습니다.


In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
!head batch_data.csv.out

### 마무리

축하합니다! Amazon SageMaker를 사용하여 배치 변환 작업을 실행했습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 **Conclusion** 부분을 계속 진행합니다.